# Discrete Fourier Transform

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

print("Input Sine Wave Signal:")
N = 360 # degrees (Number of samples)
a = random.randint(1, 100)
f = random.randint(1, 100)
p = random.randint(0, 360)
print("frequency = {:3d}".format(f))
print("amplitude = {:3d}".format(a))
print("phase ang = {:3d}".format(p))

## Manually calculate the inverse Fourier transform

We can calculate the sine wave with $A\sin(ft + \phi)$. Because we are working in degrees, and Python's `sin()` function only understands radians, we need to convert using the `radians()` function.

In [ ]:
from math import sin, pi, radians

f_list = []
for n in range(N):
    sample = a * sin(radians(f * n + p))
    f_list.append(sample)

In [ ]:
plt.plot(f_list)

We're going to implement the Fourier transform as given on Wolfram Alpha:
$$
F_n = \sum_{k=0}^{N-1} f_k e^{-2\pi ink/N}.
$$

In [ ]:
from numpy import exp

#function to calculate the Discrete Fourier Transform
def DFT(f_list):
    N = len(f_list)
    DFT_list = []
    for n in range(N):
        Fn = 0.0
        for k in range(N):
            Fn += f_list[k] * exp(-2j * pi * n * k / N)
        DFT_list.append(Fn)
    return DFT_list        

In [ ]:
DFT_list = DFT(f_list)

In [ ]:
for n, coefficient in enumerate(DFT_list[:20]):
    print("F_{:<3d} = {}".format(n, coefficient))

In [ ]:
plt.plot(np.absolute(DFT_list))

We're going to implement the *inverse* Fourier transform as given on Wolfram Alpha:
$$
f_k = \frac{1}{N} \sum_{n=0}^{N-1} F_n e^{2\pi ikn/N}.
$$

In [ ]:
#function to calculate the inverse Fourier transform
def inverse_DFT(DFT_list):
    N = len(DFT_list)
    f_list = []
    for k in range(N):
        fk = 0.0
        for n in range(N):
            fk += DFT_list[n] * exp(2j * pi * k * n / N)
        f_list.append(fk / N)
    return f_list        

In [ ]:
f_list2 = inverse_DFT(DFT_list)

In [ ]:
for n, (coefficient1, coefficient2) in enumerate(zip(f_list, f_list2[:10])):
    print("f_{:<3d} = {}\t{}".format(n, coefficient1, coefficient2))

Unfortunately, due to rounding errors, we're still seeing small imaginary values. They really are small. Let's calculate the largest absolute value of the imaginary parts:

In [ ]:
np.max(np.abs(np.imag(f_list2)))

So, indeed, rounding errors. Let's make `f_list2` real by ignoring the imaginary part:

In [ ]:
f_list2 = [np.real(u) for u in f_list2]

In [ ]:
for n, (coefficient1, coefficient2) in enumerate(zip(f_list, f_list2[:10])):
    print("f_{:<3d} = {}\t{}".format(n, coefficient1, coefficient2))

Let's see if the original and recreated values are the same:

In [ ]:
plt.plot(f_list)
plt.plot(f_list2)

They are! The plotted lines coincide.

## Again, but now with some NumPy magic

Calculating the original samples:

In [ ]:
n = np.arange(N)
f_list = a * np.sin(np.radians(f * n + p))

In [ ]:
plt.plot(f_list)